In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/StockForcasting/Stock-Forcast')

In [16]:
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.layers import LSTM, Dense, Softmax, BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from functions import *
import gc

In [4]:
import os
os.chdir('/content/drive/MyDrive/dataForStockForcast')

In [5]:
num_features = 4 # Number of features in the data (Example: Low, High, etc)
time_steps = 10 # Time steps to consider
num_stocks = 30 # The dataset is a list of 2000 stock dataframes. num_stocks is the number of stocks to consider for training (Matter of RAM capacity)

In [6]:
units = 64
model = Sequential((LSTM(units, input_shape=(time_steps, 4), return_sequences=True),
                   LSTM(2*units, return_sequences=False),
                   Dense(256),
                   Dense(128),
                   Dense(3),
                   Softmax()))
model.compile(loss=CategoricalCrossentropy(), optimizer=Adam(learning_rate=0.001), metrics='Accuracy')
callbacks = [EarlyStopping(patience=15, restore_best_weights=True, min_delta=1e-4), ReduceLROnPlateau(verbose=True)]

In [7]:
x_train, x_valid, y_train, y_valid = preprocess_data_equal_division('training_set.pkl', num_stocks=num_stocks, time_steps=time_steps, equal_split=False)

In [9]:
model.compile(loss=CategoricalCrossentropy(), optimizer=Adam(learning_rate=0.001), metrics='Accuracy')
callbacks = [EarlyStopping(patience=15, restore_best_weights=True, min_delta=1e-4), ReduceLROnPlateau(verbose=True)]
model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=10, batch_size=32)

Epoch 1/10
1644/1644 [==============================] - 49s 28ms/step - loss: 0.7819 - Accuracy: 0.4923 - val_loss: 0.7764 - val_Accuracy: 0.4976
Epoch 2/10
1644/1644 [==============================] - 45s 27ms/step - loss: 0.7811 - Accuracy: 0.4912 - val_loss: 0.7771 - val_Accuracy: 0.4938
Epoch 3/10
1644/1644 [==============================] - 44s 27ms/step - loss: 0.7809 - Accuracy: 0.4916 - val_loss: 0.7756 - val_Accuracy: 0.5014
Epoch 4/10
1644/1644 [==============================] - 42s 25ms/step - loss: 0.7808 - Accuracy: 0.4946 - val_loss: 0.7759 - val_Accuracy: 0.4983
Epoch 5/10
1644/1644 [==============================] - 42s 25ms/step - loss: 0.7807 - Accuracy: 0.4955 - val_loss: 0.7764 - val_Accuracy: 0.4972
Epoch 6/10
1644/1644 [==============================] - 43s 26ms/step - loss: 0.7805 - Accuracy: 0.4923 - val_loss: 0.7764 - val_Accuracy: 0.4909
Epoch 7/10
1644/1644 [==============================] - 44s 27ms/step - loss: 0.7806 - Accuracy: 0.4931 - val_loss: 0.7766 -

In [10]:
model.save('model_lstm.keras')

In [29]:
_a, _p, _r, _s, _f = metric_calculations_categorical(model, x_train, y_train, set_="Training set")

1644/1644 [==============================] - 15s 9ms/step
For Training set
  Accuracy: 49.711070559610704,
  Precision: 49.42637759710931,
  Recall: 41.76398748187161,
  Specificity: 57.59675831250474,
  F1score: 45.27326134624136


In [30]:
# del(x_train)
# del(y_train)
_, _, _, _, _ = metric_calculations_categorical(model, x_valid, y_valid, set_="Validation set")
# del(x_valid)
# del(y_valid)

411/411 [==============================] - 3s 8ms/step
For Validation set
  Accuracy: 49.711070559610704,
  Precision: 49.87146529562982,
  Recall: 41.15151515151515,
  Specificity: 58.333333333333336,
  F1score: 45.093807072887266


In [23]:
gc.collect()

653

In [24]:
del(x_test)
del(y_test)

In [31]:
x_test, y_test = preprocess_data_equal_division('testing_set1.pkl', split=False, num_stocks=50, equal_split=False)

In [32]:
_, _, _, _, _ = metric_calculations_categorical(model, x_test, y_test, set_="Test set")

1374/1374 [==============================] - 13s 9ms/step
For Test set
  Accuracy: 48.714448236632535,
  Precision: 48.26815642458101,
  Recall: 31.35547087642896,
  Specificity: 66.18278097324934,
  F1score: 38.01561984380156
